# Full model code

This notebook contains the **final** model code and docstrings generated by Perplexity.AI between November and December 2024.

The code is stroed in a module called `ccu_formatted_code.py`. It has been autoformatted using the tool `black`. This was used to aid PEP8 compliance of the code and help with line wrapping.

In [1]:
from IPython.display import Code

## Code stored in `ccu_formatted_code.py`

In [2]:
Code(filename='ccu_formatted_code.py', language='python')

import streamlit as st
import simpy
import numpy as np
import pandas as pd


class Experiment:
    def __init__(
        self,
        interarrival_means=[22.72, 26.0, 37.0, 47.2, 575.0, 17.91],
        stay_distributions=[
            (128.79, 267.51),
            (177.89, 276.54),
            (140.15, 218.02),
            (212.86, 457.67),
            (87.53, 108.67),
            57.34,
        ],
        elective_treatment_mean=57.34,
        num_critical_care_beds=24,
        intensive_cleaning_duration=5,
        warm_up_period=30 * 24,
        results_collection_period=12 * 30 * 24,
        trace=False,
        random_number_set=0,
    ):
        self.interarrival_means = interarrival_means
        self.stay_distributions = stay_distributions
        self.elective_treatment_mean = elective_treatment_mean
        self.num_critical_care_beds = num_critical_care_beds
        self.intensive_cleaning_duration = intensive_cleaning_duration
        self.warm_up_period = warm_up_period
        self.results_collection_period = results_collection_period
        self.total_treatment_time = 0
        self.cancelled_elective_count = 0
        self.mean_waiting_time_unplanned = 0
        self.total_unplanned_admissions = 0
        self.bed_occupancy = 0
        self.trace = trace
        self.patient_count = 0
        self.random_number_set = random_number_set
        self.streams = []

        self.setup_streams(random_number_set)

    def reset_kpi(self):
        self.total_treatment_time = 0
        self.cancelled_elective_count = 0
        self.mean_waiting_time_unplanned = 0
        self.total_unplanned_admissions = 0
        self.bed_occupancy = 0
        self.patient_count = 0

    def setup_streams(self, random_number_set):
        self.streams = []
        rng = np.random.default_rng(random_number_set)
        seeds = rng.integers(0, np.iinfo(np.int64).max, size=12)
        for seed in seeds:
            self.streams.append(np.random.default_rng(seed))


class CCUModel:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.patient_count = 0
        self.critical_care_beds = simpy.Resource(
            env, capacity=experiment.num_critical_care_beds
        )

    def patient_arrival_AE(self):
        while True:
            yield self.env.timeout(
                self.experiment.streams[0].exponential(
                    self.experiment.interarrival_means[0]
                )
            )
            self.patient_count += 1
            if self.experiment.trace:
                print(
                    f"Patient {self.patient_count} arrived from Accident and Emergency at time {self.env.now}"
                )
            self.env.process(
                self.unplanned_admission(self.experiment.stay_distributions[0])
            )

    def patient_arrival_wards(self):
        while True:
            yield self.env.timeout(
                self.experiment.streams[1].exponential(
                    self.experiment.interarrival_means[1]
                )
            )
            self.patient_count += 1
            if self.experiment.trace:
                print(
                    f"Patient {self.patient_count} arrived from the Wards at time {self.env.now}"
                )
            self.env.process(
                self.unplanned_admission(self.experiment.stay_distributions[1])
            )

    def patient_arrival_surgery(self):
        while True:
            yield self.env.timeout(
                self.experiment.streams[2].exponential(
                    self.experiment.interarrival_means[2]
                )
            )
            self.patient_count += 1
            if self.experiment.trace:
                print(
                    f"Patient {self.patient_count} arrived from Emergency surgery at time {self.env.now}"
                )
            self.env.process(
                self.unplanned_admission(self.experiment.stay_distributions[2])
     